<a href="https://colab.research.google.com/github/MohammedYaseen97/END-3.0-Assignment-7/blob/main/END3_0_Assignment_7_Task_1_Precision%2C_Recall%2C_F1_Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from torchtext.datasets import IMDB, YelpReviewPolarity

In [3]:
help(IMDB)

Help on function IMDB in module torchtext.datasets.imdb:

IMDB(root='.data', split=('train', 'test'))
    IMDB dataset
    
    Separately returns the train/test split
    
    Number of lines per split:
        train: 25000
    
        test: 25000
    
    
    Number of classes
        2
    
    
    Args:
        root: Directory where the datasets are saved.
            Default: .data
        split: split or splits to be returned. Can be a string or tuple of strings.
            Default: ('train', 'test')



In [4]:
help(YelpReviewPolarity)

Help on function YelpReviewPolarity in module torchtext.datasets.yelpreviewpolarity:

YelpReviewPolarity(root='.data', split=('train', 'test'))
    YelpReviewPolarity dataset
    
    Separately returns the train/test split
    
    Number of lines per split:
        train: 560000
    
        test: 38000
    
    
    Number of classes
        2
    
    
    Args:
        root: Directory where the datasets are saved.
            Default: .data
        split: split or splits to be returned. Can be a string or tuple of strings.
            Default: ('train', 'test')



In [5]:
task = "Yelp"

In [6]:
data_train_iter = IMDB(split = 'train') if task == "IMDB" else YelpReviewPolarity(split = 'train')

In [7]:
next(data_train_iter)

(1,
 "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars.")

## Preprocessing
1. Gotta tokenize and transform the text.
2. Gotta convert the label to integer (0 or 1).

In [8]:
from torchtext.data.utils import get_tokenizer

help(get_tokenizer)

Help on function get_tokenizer in module torchtext.data.utils:

get_tokenizer(tokenizer, language='en')
    Generate tokenizer function for a string sentence.
    
    Args:
        tokenizer: the name of tokenizer function. If None, it returns split()
            function, which splits the string sentence by space.
            If basic_english, it returns _basic_english_normalize() function,
            which normalize the string first and split by space. If a callable
            function, it will return the function. If a tokenizer library
            (e.g. spacy, moses, toktok, revtok, subword), it returns the
            corresponding library.
        language: Default en
    
    Examples:
        >>> import torchtext
        >>> from torchtext.data import get_tokenizer
        >>> tokenizer = get_tokenizer("basic_english")
        >>> tokens = tokenizer("You can now install TorchText using pip!")
        >>> tokens
        >>> ['you', 'can', 'now', 'install', 'torchtext', 'using

In [9]:
tokenizer = get_tokenizer("basic_english")

In [10]:
from torchtext.vocab import build_vocab_from_iterator

help(build_vocab_from_iterator)

Help on function build_vocab_from_iterator in module torchtext.vocab.vocab_factory:

build_vocab_from_iterator(iterator: Iterable, min_freq: int = 1, specials: Union[List[str], NoneType] = None, special_first: bool = True) -> torchtext.vocab.vocab.Vocab
    Build a Vocab from an iterator.
    
    Args:
        iterator: Iterator used to build Vocab. Must yield list or iterator of tokens.
        min_freq: The minimum frequency needed to include a token in the vocabulary.
        specials: Special symbols to add. The order of supplied tokens will be preserved.
        special_first: Indicates whether to insert symbols at the beginning or at the end.
    
    
    Returns:
        torchtext.vocab.Vocab: A `Vocab` object
    
    Examples:
        >>> #generating vocab from text file
        >>> import io
        >>> from torchtext.vocab import build_vocab_from_iterator
        >>> def yield_tokens(file_path):
        >>>     with io.open(file_path, encoding = 'utf-8') as f:
        >>> 

In [11]:
def yield_tokens(data_iter):
  for _, text in data_iter:
    yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(data_train_iter), specials=["<unk>"])

In [12]:
help(vocab)

Help on Vocab in module torchtext.vocab.vocab object:

class Vocab(torch.nn.modules.module.Module)
 |  Vocab(vocab)
 |  
 |  Base class for all neural network modules.
 |  
 |  Your models should also subclass this class.
 |  
 |  Modules can also contain other Modules, allowing to nest them in
 |  a tree structure. You can assign the submodules as regular attributes::
 |  
 |      import torch.nn as nn
 |      import torch.nn.functional as F
 |  
 |      class Model(nn.Module):
 |          def __init__(self):
 |              super(Model, self).__init__()
 |              self.conv1 = nn.Conv2d(1, 20, 5)
 |              self.conv2 = nn.Conv2d(20, 20, 5)
 |  
 |          def forward(self, x):
 |              x = F.relu(self.conv1(x))
 |              return F.relu(self.conv2(x))
 |  
 |  Submodules assigned in this way will be registered, and will have their
 |  parameters converted too when you call :meth:`to`, etc.
 |  
 |  :ivar training: Boolean represents whether this module is in tr

In [13]:
vocab.set_default_index(vocab["<unk>"])

In [14]:
vocab(['My', 'name', 'is', 'Yaseen'])
vocab(["<unk>"])

[0]

In [15]:
text_pipeline = lambda x : vocab(tokenizer(x))
label_pipeline = {}
label_pipeline["IMDB"] = lambda x : int(0) if x == 'neg' else int(1)
label_pipeline["Yelp"] = lambda x : int(x)-1

In [16]:
text_pipeline("My name is Yaseen")

[17, 450, 14, 0]

In [17]:
from torch.utils.data import DataLoader

def collate_batch(batch):
  label_list, text_list, offsets = [], [], [0]
  for _label, _text in batch:
    label_list.append(label_pipeline[task](_label))
    processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
    text_list.append(processed_text)
    offsets.append(processed_text.size(0))
  label_list = torch.tensor(label_list, dtype=torch.int64)
  offsets = torch.tensor(offsets[:-1]).cumsum(dim = 0)
  text_list = torch.cat(text_list)
  return label_list.to(device), text_list.to(device), offsets.to(device)

my_loader = DataLoader(data_train_iter, batch_size = 8, shuffle = False, collate_fn = collate_batch)

In [18]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [19]:
data_train_iter = IMDB(split = 'train') if task == "IMDB" else YelpReviewPolarity(split = 'train')
num_class = len(set([label for (label, text) in data_train_iter]))
print(num_class)
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

2


In [20]:
print(model)

TextClassificationModel(
  (embedding): EmbeddingBag(464399, 64, mode=mean)
  (fc): Linear(in_features=64, out_features=2, bias=True)
)


In [21]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 100 if task == "IMDB" else 2000
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
      optimizer.zero_grad()
      predited_label = model(text, offsets)
      loss = criterion(predited_label, label)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1) # disuccees
      optimizer.step()
      total_acc += (predited_label.argmax(1) == label).sum().item()
      total_count += label.size(0)
      if idx % log_interval == 0 and idx > 0:
          elapsed = time.time() - start_time
          print('| epoch {:3d} | {:5d}/{:5d} batches '
                '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                            total_acc/total_count))
          total_acc, total_count = 0, 0
          start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    true_pos, actually_pos, predicted_pos = 0, 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()

            predicted_classes = torch.argmax(predited_label, dim=1) == 1
            # print(predicted_classes)
            # print(predicted_classes.shape)
            target_classes = label
            actually_pos += torch.sum(target_classes == 1).float()
            predicted_pos += torch.sum(predicted_classes).float()
            true_pos += torch.sum(
                  torch.tensor([predicted_classes[i] == target_classes[i] and 
                  predicted_classes[i] == 1 for i in range(len(label))]).to(device)
                ).float()

            total_count += label.size(0)
    
    accuracy = total_acc / total_count
    precision = true_pos / predicted_pos
    recall = true_pos / actually_pos
    f1 = 2 * (precision * recall) / (precision + recall)
    return accuracy, precision, recall, f1

In [22]:
data_iter = IMDB() if task=="IMDB" else YelpReviewPolarity()

In [23]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training
  
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = data_iter
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle = True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val, prec_val, rec_val, f1_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} | valid precision {:8.3f} | '
          'valid recall {:8.3f} | valid f1-score {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val, prec_val, rec_val, f1_val))
    print('-' * 59)

| epoch   1 |  2000/ 8313 batches | accuracy    0.853
| epoch   1 |  4000/ 8313 batches | accuracy    0.899
| epoch   1 |  6000/ 8313 batches | accuracy    0.905
| epoch   1 |  8000/ 8313 batches | accuracy    0.911
-----------------------------------------------------------
| end of epoch   1 | time: 82.39s | valid accuracy    0.916 | valid precision    0.945 | valid recall    0.885 | valid f1-score    0.914 
-----------------------------------------------------------
| epoch   2 |  2000/ 8313 batches | accuracy    0.916
| epoch   2 |  4000/ 8313 batches | accuracy    0.916
| epoch   2 |  6000/ 8313 batches | accuracy    0.917
| epoch   2 |  8000/ 8313 batches | accuracy    0.918
-----------------------------------------------------------
| end of epoch   2 | time: 83.31s | valid accuracy    0.924 | valid precision    0.912 | valid recall    0.941 | valid f1-score    0.926 
-----------------------------------------------------------
| epoch   3 |  2000/ 8313 batches | accuracy    0.92

In [24]:
print('Checking the results of test dataset.')
accu_test, prec_test, rec_test, f1_test = evaluate(test_dataloader)
print('test accuracy {:8.3f} | test precision {:8.3f} | '
      'test recall {:8.3f} | test f1-score {:8.3f} '.format(accu_test, 
                                                            prec_test, rec_test, 
                                                            f1_test))

Checking the results of test dataset.
test accuracy    0.938 | test precision    0.940 | test recall    0.936 | test f1-score    0.938 


In [25]:
if task == "IMDB":
  imdb_label = {  0: "neg",
                  1: "pos" }

  def predict(text, text_pipeline):
      with torch.no_grad():
          text = torch.tensor(text_pipeline(text))
          output = model(text, torch.tensor([0]))
          return output.argmax(1).item()

  ex_text_str = "Good movie."

  model = model.to("cpu")

  print("This is a %s review" %imdb_label[predict(ex_text_str, text_pipeline)])

elif task == "Yelp":

  def predict(text, text_pipeline):
      with torch.no_grad():
          text = torch.tensor(text_pipeline(text))
          output = model(text, torch.tensor([0]))
          return output.argmax(1).item() + 1

  ex_text_str = "Good product."

  model = model.to("cpu")

  print("Predicted category class : %s " %str(predict(ex_text_str, text_pipeline)))

Predicted category class : 2 
